<a href="https://colab.research.google.com/github/sergioar3143/AprendizajeProfundo/blob/main/Clasificacion_grupo_etario_transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
from scipy.signal import correlate2d, convolve2d
import pandas as pd
import torch
from torchvision import transforms
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import os
import PIL
import copy
from torch import nn
from torch.optim import Adam
from torch.optim import SGD
from sklearn.model_selection import train_test_split
import torchvision as tv

##Audio
from torchaudio.datasets import SPEECHCOMMANDS
from tqdm.auto import trange
import collections
import torchaudio.transforms as T
import librosa
import random
import IPython as ip

# Por reproducibilidad
torch.manual_seed(42)
np.random.seed(42)

!pip install gdown
import gdown
import os

URL_trainval="https://drive.google.com/uc?id=1Z1RqRo0_JiavaZw2yzZG6WETdZQ8qX86"
gdown.download(URL_trainval, "./fairface-img-margin025-trainval.zip", quiet=False)

URL_train_csv="https://drive.google.com/uc?id=1i1L3Yqwaio7YSOCj7ftgk8ZZchPG7dmH"
gdown.download(URL_train_csv, "./fairface_label_train.csv", quiet=False)

URL_val_csv="https://drive.google.com/uc?id=1wOdja-ezstMEp81tX1a-EYkFebev4h7D"
gdown.download(URL_val_csv, "./fairface_label_val.csv", quiet=False)

if not os.path.exists("./FairFace"):
    !unzip "/content/fairface-img-margin025-trainval.zip" -d "/content/FairFace"

Downloading...
From: https://drive.google.com/uc?id=1Z1RqRo0_JiavaZw2yzZG6WETdZQ8qX86
To: /content/fairface-img-margin025-trainval.zip
100%|██████████| 578M/578M [00:05<00:00, 107MB/s]
Downloading...
From: https://drive.google.com/uc?id=1i1L3Yqwaio7YSOCj7ftgk8ZZchPG7dmH
To: /content/fairface_label_train.csv
100%|██████████| 3.79M/3.79M [00:00<00:00, 79.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=1wOdja-ezstMEp81tX1a-EYkFebev4h7D
To: /content/fairface_label_val.csv
100%|██████████| 448k/448k [00:00<00:00, 19.2MB/s]


Se han truncado las últimas 5000 líneas del flujo de salida.
  inflating: /content/FairFace/val/5370.jpg  
  inflating: /content/FairFace/val/5596.jpg  
  inflating: /content/FairFace/val/5875.jpg  
  inflating: /content/FairFace/val/5878.jpg  
  inflating: /content/FairFace/val/5879.jpg  
  inflating: /content/FairFace/val/5883.jpg  
  inflating: /content/FairFace/val/6126.jpg  
  inflating: /content/FairFace/val/6132.jpg  
  inflating: /content/FairFace/val/6144.jpg  
  inflating: /content/FairFace/val/6150.jpg  
  inflating: /content/FairFace/val/6340.jpg  
  inflating: /content/FairFace/val/7190.jpg  
  inflating: /content/FairFace/val/7734.jpg  
  inflating: /content/FairFace/val/7747.jpg  
  inflating: /content/FairFace/val/7872.jpg  
  inflating: /content/FairFace/val/7940.jpg  
  inflating: /content/FairFace/val/8065.jpg  
  inflating: /content/FairFace/val/8069.jpg  
  inflating: /content/FairFace/val/8125.jpg  
  inflating: /content/FairFace/val/8202.jpg  
  inflating: /conte

In [2]:
df_train = pd.read_csv('/content/fairface_label_train.csv')
df_val = pd.read_csv('/content/fairface_label_val.csv')
df_train.head()

,file,age,gender,race,service_test
0,train/1.jpg,50-59,Male,East Asian,True
1,train/2.jpg,30-39,Female,Indian,False
2,train/3.jpg,3-9,Female,Black,False
3,train/4.jpg,20-29,Female,Indian,True
4,train/5.jpg,20-29,Female,Indian,True
